# Deep Learning with Torch - Tutorial 2
## Optimization library
We are interested to use better optimizers. To do so we will use the `optim` library
This package contains several optimization routines for Torch.

Each optimization algorithm is based on the same interface:

```lua
x*, {f}, ... = optim.method(func, x, state)
```

where:

* `func`: a user-defined closure that respects this API: `f, df/dx = func(x)`
* `x`: the current parameter vector (a 1D `torch.Tensor`)
* `state`: a table of parameters, and state variables, dependent upon the algorithm
* `x*`: the new parameter vector that minimizes `f, x* = argmin_x f(x)`
* `{f}`: a table of all f values, in the order they've been evaluated
  (for some simple algorithms, like SGD, `#f == 1`)
  
#### Available algorithms

Please check [this file](doc/index.md) for the full list of
optimization algorithms available and examples. 

#### Important Note

The state table is used to hold the state of the algorithm.
It's usually initialized once, by the user, and then passed to the optim function
as a black box. Example:

```lua
state = {
   learningRate = 1e-3,
   momentum = 0.5
}

for i,sample in ipairs(training_samples) do
    local func = function(x)
       -- define eval function
       return f,df_dx
    end
    optim.sgd(func,x,state)
end
```

### Preparing the data
We will again use the `MNIST` dataset, this time trying to classify correctly the digit in each sample.

A good first step will be to normalize our data to have zero mean, unit variance

In [113]:
local mnist = require 'mnist';
trainData = mnist.traindataset().data:float();
trainLabels = mnist.traindataset().label:add(1);
testData = mnist.testdataset().data:float();
testLabels = mnist.testdataset().label:add(1);

--We'll start by normalizing our data
mean = trainData:mean()
std = trainData:std()
trainData:add(-mean):div(std); 
testData:add(-mean):div(std);

In [114]:
itorch.image(trainData:narrow(1,1,5))
print('Labels:',trainLabels:narrow(1,1,5):totable())

Labels:	{
  1 : 6
  2 : 1
  3 : 5
  4 : 2
  5 : 10
}


### Shuffling data
Another good habit is to shuffle our data each epoch. We will implement a simple utility function

In [126]:
function shuffle(data, labels) --shuffle data function
    local RandOrder = torch.randperm(data:size(1)):long()
    return data:index(1,RandOrder), labels:index(1,RandOrder)
end

## Define model and criterion
We'll create a classification model. Notice the non-linearity, and `LogSoftMax`

In [155]:
require 'nn'
require 'cunn'

local inputSize = 28*28
local outputSize = 10
local layerSize = {inputSize, 64,128,256}

model = nn.Sequential()
model:add(nn.View(28 * 28)) --reshapes the image into a vector without copy
for i=1, #layerSize-1 do
    model:add(nn.Linear(layerSize[i], layerSize[i+1]))
    model:add(nn.ReLU())
end

model:add(nn.Linear(layerSize[#layerSize], outputSize))
model:add(nn.LogSoftMax())


model:cuda() --ship to gpu
print(tostring(model))

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> output]
  (1): nn.View(784)
  (2): nn.Linear(784 -> 64)
  (3): nn.ReLU
  (4): nn.Linear(64 -> 128)
  (5): nn.ReLU
  (6): nn.Linear(128 -> 256)
  (7): nn.ReLU
  (8): nn.Linear(256 -> 10)
  (9): nn.LogSoftMax
}	


In [156]:
w, dE_dw = model:getParameters()
print('Number of parameters:', w:nElement()) --over-specified model

Number of parameters:	94154	


### Classification criterion
Since we are now interested in classification, we will use the negative-log-likelihood criterion

In [10]:
criterion = nn.ClassNLLCriterion():cuda()

### Some predefined constants

In [136]:
require 'optim'
batchSize = 128

optimState = {
    learningRate = 0.1
    
}

### Main evaluation + training function

In [127]:
function forwardNet(data, train)
    --another helpful function of optim is ConfusionMatrix
    local confusion = optim.ConfusionMatrix(torch.range(0,9):totable())
    local lossAcc = 0
    local numBatches = 0
    if train then
        --set network into training mode
        model:training()
    end
    for i = 1, data:size(1) - batchSize, batchSize do
        numBatches = numBatches + 1
        local x = data:narrow(1, i, batchSize):cuda()
        local yt = data:narrow(1, i, batchSize):cuda()
        local y = model:forward(x)
        local err = criterion:forward(y, yt)
        lossAcc = lossAcc + err
        confusion:batchAdd(y,yt)
        
        if train then
            function feval()
                model:zeroGradParameters() --zero grads
                local dE_dy = criterion:backward(y,yt)
                model:backward(x, dE_dy) -- backpropagation
            
                return err, dE_dw
            end
        
            optim.sgd(feval, w, optimState)
        end
    end
    
    confusion:updateValids()
    local avgLoss = lossAcc / numBatches
    local avgError = 1 - confusion.totalValid
    
    return avgLoss, avgError, tostring(confusion)
end

### Train the network on training set, evaluate on seperate set

In [157]:

epochs = 25
trainLoss = torch.Tensor(epochs)
testLoss = torch.Tensor(epochs)
trainError = torch.Tensor(epochs)
testError = torch.Tensor(epochs)

--reset net weights
model:apply(function(l) l:reset() end)

for e = 1, epochs do
    trainData, trainLabels = shuffle(trainData, trainLabels) --shuffle training data
    trainLoss[e], trainError[e] = forwardNet(trainData, trainLabels, true)
    testLoss[e], testError[e], confusion = forwardNet(testData, testLabels, false)
    
    if e % 5 == 0 then
        print('Epoch ' .. e .. ':')
        print('Training error: ' .. trainError[e], 'Training Loss: ' .. trainLoss[e])
        print('Test error: ' .. testError[e], 'Test Loss: ' .. testLoss[e])
        print(confusion)
    end
end



Epoch 5:	
Training error: 0.024272168803419	Training Loss: 0.077327728828686	
Test error: 0.026642628205128	Test Loss: 0.084720106365589	
ConfusionMatrix:
[[     971       0       1       0       1       1       0       1       2       2]   99.183% 	[class: 0]
 [       0    1118       4       1       0       1       2       1       6       0]   98.676% 	[class: 1]
 [       9       1    1005       1       6       0       1       5       2       0]   97.573% 	[class: 2]
 [       0       0       8     967       0      14       0       9       6       4]   95.933% 	[class: 3]
 [       1       0       2       0     957       0       3       2       1      14]   97.653% 	[class: 4]
 [       4       1       1       4       3     866       5       1       3       2]   97.303% 	[class: 5]
 [       9       2       1       1       5       2     931       0       5       0]   97.385% 	[class: 6]
 [       1       6      13       0       0       0       0     998       3       6]   97.176% 	[class: 

Epoch 10:	
Training error: 0.010700454059829	Training Loss: 0.034495561960766	
Test error: 0.025440705128205	Test Loss: 0.084857869797792	
ConfusionMatrix:
[[     962       1       1       1       1       2       5       2       1       3]   98.264% 	[class: 0]
 [       0    1122       0       2       0       1       1       1       5       1]   99.029% 	[class: 1]
 [       7       5     983      10       3       0       1       4      17       0]   95.437% 	[class: 2]
 [       0       0       1     993       0       4       0       3       4       3]   98.512% 	[class: 3]
 [       0       0       1       0     965       0       5       3       0       6]   98.469% 	[class: 4]
 [       2       1       0      14       1     856       4       1       7       4]   96.180% 	[class: 5]
 [       2       3       1       1       4       5     934       0       6       0]   97.699% 	[class: 6]
 [       0       8       4       6       0       0       0     992       7      10]   96.592% 	[class:

Epoch 15:	
Training error: 0.0050247061965812	Training Loss: 0.01641820369559	
Test error: 0.022636217948718	Test Loss: 0.08377761231401	
ConfusionMatrix:
[[     960       1       2       1       1       1       5       1       3       4]   98.059% 	[class: 0]
 [       0    1121       1       1       0       1       2       2       5       0]   98.941% 	[class: 1]
 [       6       2    1008       2       4       0       1       3       4       0]   97.864% 	[class: 2]
 [       0       0       3     986       0       6       0       5       7       1]   97.817% 	[class: 3]
 [       0       0       2       1     950       1       3       3       0      20]   96.939% 	[class: 4]
 [       1       0       0       6       2     867       4       2       7       1]   97.416% 	[class: 5]
 [       3       2       1       1       4       4     937       0       4       0]   98.013% 	[class: 6]
 [       1       5       6       0       0       0       0    1007       5       3]   98.053% 	[class: 

Epoch 20:	
Training error: 0.0013521634615384	Training Loss: 0.0057550926382343	
Test error: 0.024238782051282	Test Loss: 0.10064986976198	
ConfusionMatrix:
[[     969       1       1       0       1       1       1       2       2       1]   98.979% 	[class: 0]
 [       0    1124       1       2       0       1       1       2       1       1]   99.206% 	[class: 1]
 [       8       2     990      12       3       0       1       8       6       0]   96.117% 	[class: 2]
 [       0       0       2     993       0       4       0       6       1       2]   98.512% 	[class: 3]
 [       4       0       4       1     949       0       1       4       0      17]   96.837% 	[class: 4]
 [       3       1       1      10       3     861       2       2       3       4]   96.742% 	[class: 5]
 [      11       2       3       1       6       3     926       0       4       0]   96.862% 	[class: 6]
 [       0       3       4       1       2       0       0    1010       3       4]   98.345% 	[class

Epoch 25:	
Training error: 0.00018362713675213	Training Loss: 0.0016332661860392	
Test error: 0.021133814102564	Test Loss: 0.10056200719033	
ConfusionMatrix:
[[     969       1       1       1       1       1       3       1       0       1]   98.979% 	[class: 0]
 [       0    1121       2       2       0       1       2       1       3       1]   98.941% 	[class: 1]
 [       7       2    1008       3       3       0       1       2       4       0]   97.864% 	[class: 2]
 [       0       0       4     992       0       3       0       5       2       2]   98.413% 	[class: 3]
 [       1       0       4       1     952       1       2       4       1      14]   97.143% 	[class: 4]
 [       2       1       0      11       2     861       4       2       4       3]   96.742% 	[class: 5]
 [       4       2       1       1       2       1     941       0       4       0]   98.431% 	[class: 6]
 [       2       3       6       1       1       0       0    1004       6       4]   97.760% 	[clas

In [140]:
function plotError(trainError, testError, title)
    local range = torch.range(1, trainError:size(1))
    local plot = itorch.Plot()
    plot:line(range, trainError * 100, 'red', 'Training error')
    plot:line(range, testError * 100, 'blue', 'Test error'):draw()
    plot:title(title):redraw()
    plot:xaxis('epoch'):yaxis('% error'):redraw()
    plot:legend(true)
    plot:redraw()
end

plotError(trainError, testError, 'Classification Error')

### Introduce momentum, L2 regularization

In [148]:
--reset net weights
model:apply(function(l) l:reset() end)

optimState = {
    learningRate = 0.1,
    momentum = 0.9,
    weightDecay = 1e-3
    
}
for e = 1, epochs do
    trainData, trainLabels = shuffle(trainData, trainLabels) --shuffle training data
    trainLoss[e], trainError[e] = forwardNet(trainData, trainLabels, true)
    testLoss[e], testError[e], confusion = forwardNet(testData, testLabels, false)
end

print('Training error: ' .. trainError[epochs], 'Training Loss: ' .. trainLoss[epochs])
print('Test error: ' .. testError[epochs], 'Test Loss: ' .. testLoss[epochs])
plotError(trainError, testError, 'Classification Error')

Training error: 0.0050914797008547	Training Loss: 0.020095667451556	
Test error: 0.019831730769231	Test Loss: 0.066589443610074	


### Insert a Dropout layer

In [153]:
model:insert(nn.Dropout(0.9):cuda(), 8)

print(tostring(model))

--reset net weights
model:apply(function(l) l:reset() end)

optimState = {
    learningRate = 0.1,
    momentum = 0.9
    
}
for e = 1, epochs do
    trainData, trainLabels = shuffle(trainData, trainLabels) --shuffle training data
    trainLoss[e], trainError[e] = forwardNet(trainData, trainLabels, true)
    testLoss[e], testError[e], confusion = forwardNet(testData, testLabels, false)
end

print('Training error: ' .. trainError[epochs], 'Training Loss: ' .. trainLoss[epochs])
print('Test error: ' .. testError[epochs], 'Test Loss: ' .. testLoss[epochs])
plotError(trainError, testError, 'Classification Error')

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> output]
  (1): nn.View(784)
  (2): nn.Linear(784 -> 64)
  (3): nn.ReLU
  (4): nn.Linear(64 -> 128)
  (5): nn.ReLU
  (6): nn.Linear(128 -> 256)
  (7): nn.ReLU
  (8): nn.Dropout(0.900000)
  (9): nn.Linear(256 -> 10)
  (10): nn.LogSoftMax
}	


Training error: 0.016276041666667	Training Loss: 0.050709126298276	
Test error: 0.031650641025641	Test Loss: 0.17221213516612	


### Change optimization algorithm, learning-rate, layer sizes...